## Dependencies

In [28]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
from scipy import stats

In [74]:
df = pd.read_excel('results.xlsx')

df['label'] = df['label'].map(lambda x:x.replace('give_me_some_credit','give-me-some-credit'))
df['label'] = df['label'].map(lambda x:x.replace('lendinc_club','lending-club'))

df[['method','dataset','sample']] = df['label'].map(lambda x:x.split('_')).apply(pd.Series).dropna()

In [75]:
df['method'] = df['method'].map({'uni':'EQD','qua':'EQF','kme':'KME','gau':'GMM','dcc':'DCC','dec':'DEC'})

## Overall Performance

### AUC

In [84]:
auc = []
for m,d in [(m,d['roc']) for m,d in df.loc[df['sample']=='validate'].groupby('method')]:
    interval = stats.norm.interval(alpha=0.95,loc=d.mean(),scale=stats.sem(d))
    #interval = stats.t.interval(alpha=0.95,df=len(d)-1,loc=d.mean(),scale=stats.sem(d))
    interval = '['+",".join(map(str,map(lambda x:round(x,4),interval)))+']'
    auc.append((m,d.mean().round(4),interval,len(d)))
auc = pd.DataFrame(auc,columns=['Method','Mean','95% CI','n'])
display(auc)
print(auc.drop('n',axis=1).to_latex(index=False))

,Method,Mean,95% CI,n
0,DCC,0.7678,"[0.7587,0.777]",330
1,DEC,0.7666,"[0.7572,0.776]",330
2,EQD,0.5295,"[0.5237,0.5354]",330
3,EQF,0.7510,"[0.7414,0.7607]",330
4,GMM,0.6307,"[0.6235,0.6379]",330
5,KME,0.5742,"[0.5657,0.5828]",330


\begin{tabular}{lrl}
\toprule
Method &   Mean &          95\% CI \\
\midrule
   DCC & 0.7678 &  [0.7587,0.777] \\
   DEC & 0.7666 &  [0.7572,0.776] \\
   EQD & 0.5295 & [0.5237,0.5354] \\
   EQF & 0.7510 & [0.7414,0.7607] \\
   GMM & 0.6307 & [0.6235,0.6379] \\
   KME & 0.5742 & [0.5657,0.5828] \\
\bottomrule
\end{tabular}



/tmp/ipykernel_931667/1126205161.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(auc.drop('n',axis=1).to_latex(index=False))


### KS

In [85]:
ks = []
for m,d in [(m,d['ks']) for m,d in df.loc[df['sample']=='validate'].groupby('method')]:
    interval = stats.norm.interval(alpha=0.95,loc=d.mean(),scale=stats.sem(d))
    #interval = stats.t.interval(alpha=0.95,df=len(d)-1,loc=d.mean(),scale=stats.sem(d))
    interval = '['+",".join(map(str,map(lambda x:round(x,4),interval)))+']'
    ks.append((m,d.mean().round(4),interval))
ks = pd.DataFrame(ks,columns=['Method','Mean','95% CI'])
display(ks)
print(ks.to_latex(index=False))

,Method,Mean,95% CI
0,DCC,0.3891,"[0.3702,0.4079]"
1,DEC,0.3871,"[0.3684,0.4057]"
2,EQD,0.0381,"[0.0299,0.0462]"
3,EQF,0.3661,"[0.3474,0.3847]"
4,GMM,0.1814,"[0.1692,0.1936]"
5,KME,0.1144,"[0.1009,0.128]"


\begin{tabular}{lrl}
\toprule
Method &   Mean &          95\% CI \\
\midrule
   DCC & 0.3891 & [0.3702,0.4079] \\
   DEC & 0.3871 & [0.3684,0.4057] \\
   EQD & 0.0381 & [0.0299,0.0462] \\
   EQF & 0.3661 & [0.3474,0.3847] \\
   GMM & 0.1814 & [0.1692,0.1936] \\
   KME & 0.1144 &  [0.1009,0.128] \\
\bottomrule
\end{tabular}



/tmp/ipykernel_931667/2228313384.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ks.to_latex(index=False))


### CPU Time

In [86]:
cpu = []
for m,d in [(m,d['completion_time']) for m,d in df.loc[df['sample']=='validate'].groupby('method')]:
    d/=1000
    interval = stats.norm.interval(alpha=0.95,loc=d.mean(),scale=stats.sem(d))
    #interval = stats.t.interval(alpha=0.95,df=len(d)-1,loc=d.mean(),scale=stats.sem(d))
    interval = '['+",".join(map(str,map(lambda x:round(x,4),interval)))+']'
    cpu.append((m,d.mean().round(4),interval))
cpu = pd.DataFrame(cpu,columns=['Method','Mean','95% CI'])
display(cpu)
print(cpu.to_latex(index=False))

,Method,Mean,95% CI
0,DCC,1.1319,"[0.9224,1.3414]"
1,DEC,4.1383,"[3.3257,4.9508]"
2,EQD,1.1066,"[0.8968,1.3165]"
3,EQF,1.1469,"[0.935,1.3588]"
4,GMM,1.1777,"[0.9602,1.3951]"
5,KME,1.1777,"[0.9614,1.3939]"


\begin{tabular}{lrl}
\toprule
Method &   Mean &          95\% CI \\
\midrule
   DCC & 1.1319 & [0.9224,1.3414] \\
   DEC & 4.1383 & [3.3257,4.9508] \\
   EQD & 1.1066 & [0.8968,1.3165] \\
   EQF & 1.1469 &  [0.935,1.3588] \\
   GMM & 1.1777 & [0.9602,1.3951] \\
   KME & 1.1777 & [0.9614,1.3939] \\
\bottomrule
\end{tabular}



/tmp/ipykernel_931667/3492813642.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(cpu.to_latex(index=False))


## Wilcoxon Test

### AUC

In [87]:
aux = df.loc[df['sample']=='validate'].reset_index(drop=True)

In [91]:
l = []
for m1,d1 in [(m,d['roc']) for m,d in aux[['roc','method']].groupby('method')][:2]:
    for m2,d2 in [(m,d['roc']) for m,d in aux[['roc','method']].groupby('method')][2:]:
        res = stats.ranksums(d1,d2)
        l.append([f'{m1} vs {m2}',res.statistic,res.pvalue])
        

print(pd.DataFrame(l,columns=['Contrast','Wilcoxon','p-value']).round(4).to_latex(index=False))

\begin{tabular}{lrr}
\toprule
  Contrast &  Wilcoxon &  p-value \\
\midrule
DCC vs EQD &   21.6165 &   0.0000 \\
DCC vs EQF &    2.6339 &   0.0084 \\
DCC vs GMM &   17.6156 &   0.0000 \\
DCC vs KME &   19.7597 &   0.0000 \\
DEC vs EQD &   21.5658 &   0.0000 \\
DEC vs EQF &    2.4441 &   0.0145 \\
DEC vs GMM &   17.3016 &   0.0000 \\
DEC vs KME &   19.5782 &   0.0000 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_931667/4255282993.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(l,columns=['Contrast','Wilcoxon','p-value']).round(4).to_latex(index=False))


### KS

In [92]:
l = []
for m1,d1 in [(m,d['ks']) for m,d in aux[['ks','method']].groupby('method')][:2]:
    for m2,d2 in [(m,d['ks']) for m,d in aux[['ks','method']].groupby('method')][2:]:
        res = stats.ranksums(d1,d2)
        l.append([f'{m1} vs {m2}',res.statistic,res.pvalue])
        
print(pd.DataFrame(l,columns=['Contrast','Wilcoxon','p-value']).round(4).to_latex(index=False))

\begin{tabular}{lrr}
\toprule
  Contrast &  Wilcoxon &  p-value \\
\midrule
DCC vs EQD &   21.0800 &   0.0000 \\
DCC vs EQF &    2.0482 &   0.0405 \\
DCC vs GMM &   15.5547 &   0.0000 \\
DCC vs KME &   18.0641 &   0.0000 \\
DEC vs EQD &   21.0706 &   0.0000 \\
DEC vs EQF &    1.7730 &   0.0762 \\
DEC vs GMM &   15.2842 &   0.0000 \\
DEC vs KME &   17.9385 &   0.0000 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_931667/2036966153.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame(l,columns=['Contrast','Wilcoxon','p-value']).round(4).to_latex(index=False))


## Winning Method on each dataset

### AUC

In [95]:
ind = aux.pivot_table(columns=['method'],index='dataset',aggfunc='mean',values='roc')

ind['Best Performing Method'] = ind.idxmax(axis=1)

print(ind.round(2).to_latex())

\begin{tabular}{lrrrrrrl}
\toprule
method &   DCC &   DEC &   EQD &   EQF &   GMM &   KME & Best Performing Method \\
dataset             &       &       &       &       &       &       &                        \\
\midrule
australian          &  0.85 &  0.84 &  0.54 &  0.84 &  0.62 &  0.55 &                    DCC \\
farmers             &  0.66 &  0.67 &  0.50 &  0.66 &  0.64 &  0.52 &                    DEC \\
german              &  0.67 &  0.67 &  0.51 &  0.66 &  0.63 &  0.61 &                    DEC \\
give-me-some-credit &  0.84 &  0.84 &  0.50 &  0.81 &  0.65 &  0.64 &                    DEC \\
hmeq                &  0.91 &  0.91 &  0.51 &  0.90 &  0.64 &  0.51 &                    DEC \\
japan               &  0.84 &  0.84 &  0.50 &  0.84 &  0.59 &  0.50 &                    DCC \\
lending-club        &  0.68 &  0.66 &  0.67 &  0.66 &  0.68 &  0.67 &                    DCC \\
mexico              &  0.76 &  0.76 &  0.50 &  0.72 &  0.73 &  0.69 &                    DCC \\
mortgage 

/tmp/ipykernel_931667/2541546006.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ind.round(2).to_latex())


### KS

In [100]:
ind = aux.pivot_table(columns=['method'],index='dataset',aggfunc='mean',values='ks')

ind['Best Performing Method'] = ind.idxmax(axis=1)
display(ind.round(2))
print(ind.round(2).to_latex())

method,DCC,DEC,EQD,EQF,GMM,KME,Best Performing Method
dataset,,,,,,,
australian,0.57,0.56,0.06,0.54,0.20,0.09,DCC
farmers,0.27,0.27,0.00,0.27,0.24,0.04,DCC
german,0.26,0.26,0.01,0.24,0.21,0.17,DEC
give-me-some-credit,0.49,0.49,0.00,0.46,0.17,0.25,DEC
hmeq,0.67,0.67,0.01,0.65,0.20,0.01,DCC
japan,0.57,0.56,0.00,0.55,0.16,0.00,DCC
lending-club,0.26,0.24,0.23,0.22,0.26,0.23,DCC
mexico,0.39,0.39,0.00,0.31,0.35,0.27,DCC
mortgage,0.05,0.07,0.00,0.05,0.01,0.00,DEC


\begin{tabular}{lrrrrrrl}
\toprule
method &   DCC &   DEC &   EQD &   EQF &   GMM &   KME & Best Performing Method \\
dataset             &       &       &       &       &       &       &                        \\
\midrule
australian          &  0.57 &  0.56 &  0.06 &  0.54 &  0.20 &  0.09 &                    DCC \\
farmers             &  0.27 &  0.27 &  0.00 &  0.27 &  0.24 &  0.04 &                    DCC \\
german              &  0.26 &  0.26 &  0.01 &  0.24 &  0.21 &  0.17 &                    DEC \\
give-me-some-credit &  0.49 &  0.49 &  0.00 &  0.46 &  0.17 &  0.25 &                    DEC \\
hmeq                &  0.67 &  0.67 &  0.01 &  0.65 &  0.20 &  0.01 &                    DCC \\
japan               &  0.57 &  0.56 &  0.00 &  0.55 &  0.16 &  0.00 &                    DCC \\
lending-club        &  0.26 &  0.24 &  0.23 &  0.22 &  0.26 &  0.23 &                    DCC \\
mexico              &  0.39 &  0.39 &  0.00 &  0.31 &  0.35 &  0.27 &                    DCC \\
mortgage 

/tmp/ipykernel_931667/214429515.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ind.round(2).to_latex())
